# Исследование надёжности заёмщиков
Задача выявить зависимость признаков на факт погашения задолжености в срок. Результаты анализа будут использоваться при построении модели **кредитного скоринга**, машинного обучения.

Описание данных  
- `children` — количество детей в семье  
- `days_employed` — общий трудовой стаж в днях  
- `dob_years` — возраст клиента в годах  
- `education` — уровень образования клиента  
- `education_id` — идентификатор уровня образования  
- `family_status` — семейное положение  
- `family_status_id` — идентификатор семейного положения  
- `gender` — пол клиента  
- `income_type` — тип занятости  
- `debt` — имел ли задолженность по возврату кредитов  
- `total_income` — ежемесячный доход  
- `purpose` — цель получения кредита  

## Шаг 1. Изучение общей информации


In [1]:
import pandas as pd
from collections import Counter
from pymystem3 import Mystem
m = Mystem()

In [2]:
server_path = '/datasets/data.csv'
filename = 'data.csv'

try:
    data = pd.read_csv(server_path) #пробуем найти данные по сетевому адресу
except: 
    data = pd.read_csv(filename) #если не можем найти данные по сетевому адресу, ищем по локальному

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [6]:
data.shape

(21525, 12)

Общий итог: Кол-во строк 21525,  в двух колонках меньше количество значений - пропуски?, 12 колонок, 'days_employed' - есть отрицательные значения. В колонке "education" значения написаны в разных регистрах

## Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

    Обнаружены пропуски в двух колонках. days_employed - Общий трудовой стаж, total_income - ежемесячный доход. Скорее всего пропуск обозначает что нет данных по этим значениям. клиент не предоставил или данные не сохранились. Пропущенные значения составляют 10% от общего количества данных. 
   

In [8]:
print('пропуски в колонке days_employed до:', 
data['days_employed'].isna().sum())

пропуски в колонке days_employed до: 2174


Избавляемся от минусов в колонке. Отрицательный трудовой стаж не может быть. Скорее всего это ошибка в знаке. Получить информацию от ответсвенного за выгрузку данных не получилось.

In [9]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим уникальные значения колонок образование и семейного статуса.

In [10]:
data.groupby(['education', 'education_id'])['education', 'education_id'].count()

C:\Users\mkt4\AppData\Local\Temp/ipykernel_9028/2734662045.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data.groupby(['education', 'education_id'])['education', 'education_id'].count()


,,education,education_id
education,education_id,,
ВЫСШЕЕ,0,274,274
Высшее,0,268,268
НАЧАЛЬНОЕ,3,17,17
НЕОКОНЧЕННОЕ ВЫСШЕЕ,2,29,29
Начальное,3,15,15
Неоконченное высшее,2,47,47
СРЕДНЕЕ,1,772,772
Среднее,1,711,711
УЧЕНАЯ СТЕПЕНЬ,4,1,1


In [11]:
#Приведем значения в education к единому регистру
data['education'] = data['education'].str.lower() 

In [12]:
data.groupby(['education', 'education_id'])['education', 'education_id'].count()

C:\Users\mkt4\AppData\Local\Temp/ipykernel_9028/2734662045.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data.groupby(['education', 'education_id'])['education', 'education_id'].count()


,,education,education_id
education,education_id,,
высшее,0,5260,5260
начальное,3,282,282
неоконченное высшее,2,744,744
среднее,1,15233,15233
ученая степень,4,6,6


In [13]:
data.groupby(['family_status', 'family_status_id'])['family_status', 'family_status_id'].count()

C:\Users\mkt4\AppData\Local\Temp/ipykernel_9028/3477493686.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data.groupby(['family_status', 'family_status_id'])['family_status', 'family_status_id'].count()


,,family_status,family_status_id
family_status,family_status_id,,
Не женат / не замужем,4,2813,2813
в разводе,3,1195,1195
вдовец / вдова,2,960,960
гражданский брак,1,4177,4177
женат / замужем,0,12380,12380


In [14]:
#Для заполнения пропусков в days_employed расчитаем медиану по группе образования и семейный статус
#Для данной задачи считаю что эти группы значимы и 
median = data.groupby(['education', 'family_status']).agg({'days_employed': 'median'}).rename(
    columns = {'days_employed': 'median_days_employed'}
)

присоединяем таблицу к исходной таблице дата

In [15]:
data = data.merge(median, on = ['education', 'family_status'])

In [16]:
data[['family_status', 'education', 'days_employed', 'median_days_employed']][data['days_employed'].isna()].head()

,family_status,education,days_employed,median_days_employed
6,женат / замужем,высшее,NaN,-1443.773383
9,женат / замужем,высшее,NaN,-1443.773383
12,женат / замужем,высшее,NaN,-1443.773383
13,женат / замужем,высшее,NaN,-1443.773383
15,женат / замужем,высшее,NaN,-1443.773383


In [17]:
# значения где days_employed пусто заполняем медианой 
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']

In [18]:
print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски после: 0


Убираем пропуски по колонке total_income

In [19]:
data.groupby(['education', 'education_id'])['education', 'education_id'].count()

C:\Users\mkt4\AppData\Local\Temp/ipykernel_9028/2734662045.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data.groupby(['education', 'education_id'])['education', 'education_id'].count()


,,education,education_id
education,education_id,,
высшее,0,5260,5260
начальное,3,282,282
неоконченное высшее,2,744,744
среднее,1,15233,15233
ученая степень,4,6,6


In [20]:
#Расчитаем медиану по колонке total_income. Также по группам 'education', 'family_status'
median = median = data.groupby(['education', 'family_status']).agg({'total_income': 'median'}).rename(
    columns = {'total_income': 'median_total_income'}
)
median

median_total_income
education           family_status                             
высшее              Не женат / не замужем        170389.247495
                    в разводе                    172704.712824
                    вдовец / вдова               148683.729577
                    гражданский брак             177208.278286
                    женат / замужем              177914.294250
начальное           Не женат / не замужем        117928.036701
                    в разводе                    115007.537199
                    вдовец / вдова               103954.539610
                    гражданский брак             105533.860430
                    женат / замужем              125077.231066
неоконченное высшее Не женат / не замужем        153523.086246
                    в разводе                    195932.994615
                    вдовец / вдова               128324.048065
                    гражданский брак             162187.952277
                    женат / замужем              163208.764705
среднее             Не женат / не замужем        135824.153943
                    в разводе                    136964.250298
                    вдовец / вдова               127269.501241
                    гражданский брак             136548.656488
                    женат / замужем              137419.686830
ученая степень      Не женат / не замужем        198570.757322
                    в разводе                    115949.039788
                    женат / замужем              183408.713832

In [21]:
data = data.merge(median, on = ['education', 'family_status'])
data[['family_status', 'education', 'total_income', 'median_total_income']][data['total_income'].isna()].head()
# значения где total_income пусто заполняем медианой расчитанной по группам education family_status
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

print('Пропуски после:', data['total_income'].isna().sum())

Пропуски после: 0


### Замена типа данных

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21525 non-null  int64  
 1   days_employed         21525 non-null  float64
 2   dob_years             21525 non-null  int64  
 3   education             21525 non-null  object 
 4   education_id          21525 non-null  int64  
 5   family_status         21525 non-null  object 
 6   family_status_id      21525 non-null  int64  
 7   gender                21525 non-null  object 
 8   income_type           21525 non-null  object 
 9   debt                  21525 non-null  int64  
 10  total_income          21525 non-null  float64
 11  purpose               21525 non-null  object 
 12  median_days_employed  21525 non-null  float64
 13  median_total_income   21525 non-null  float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.5+ MB


In [23]:
#Изменяем тип данных в колонке total_income на целочисленный используем метод 
data['total_income'] = data['total_income'].astype('int')

In [24]:
data['days_employed'] = data['days_employed'].astype('int')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21525 non-null  int64  
 1   days_employed         21525 non-null  int32  
 2   dob_years             21525 non-null  int64  
 3   education             21525 non-null  object 
 4   education_id          21525 non-null  int64  
 5   family_status         21525 non-null  object 
 6   family_status_id      21525 non-null  int64  
 7   gender                21525 non-null  object 
 8   income_type           21525 non-null  object 
 9   debt                  21525 non-null  int64  
 10  total_income          21525 non-null  int32  
 11  purpose               21525 non-null  object 
 12  median_days_employed  21525 non-null  float64
 13  median_total_income   21525 non-null  float64
dtypes: float64(2), int32(2), int64(5), object(5)
memory usage: 2.3+ MB


### Обработка дубликатов

In [26]:
#Поиск дубликатов
print('Всего дубликатов в data', data.duplicated().sum())

Всего дубликатов в data 71


Обнаружено 71 одинаковая строка. Индетификаторов уникальности записи нет. (например индетификатор клиента) Количество дубликатов составляет 0,3% от всего датафрейма. Думаю можно удалить строки методом drop_duplicates. 

In [27]:
data = data.drop_duplicates()

In [28]:
print('всего дубликаторв в data после удаления', data.duplicated().sum())

всего дубликаторв в data после удаления 0


### Лемматизация

Выделим из колонки purpose (цель получения кредита) основные цели при помощи лематизации. 
Найдем уникальные значения в колонке, превратем лист с уникальными значениями в текст и после этого лематизируем слова

In [29]:
text = data['purpose'].unique()
text

array(['покупка жилья', 'операции с жильем', 'покупка недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'покупка жилья для семьи', 'операции с коммерческой недвижимостью',
       'покупка коммерческой недвижимости',
       'на покупку подержанного автомобиля', 'сделка с автомобилем',
       'жилье', 'автомобиль', 'профильное образование',
       'покупка жилья для сдачи', 'операции со своей недвижимостью',
       'автомобили', 'приобретение автомобиля',
       'на покупку своего автомобиля', 'покупка своего жилья',
       'покупка жилой недвижимости', 'заняться высшим образованием',
       'получение высшего образования',
       'сделка с подержанным автомобилем', 'ремонт жилью',
       'свой автомобиль', 'строительство жилой недвижимости',
       'высшее образование', 'строительство недвижимости',
       'операции с недвижимостью', 'получение образования', 'образование',
       'заняться образованием', 'на покупку автомобиля',
       'дополнительно

In [30]:
text = data['purpose'].unique()
purpose_text = " ".join(text)
lemas = m.lemmatize(purpose_text)
print(Counter(lemas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'строительство': 3, 'высокий': 3, 'получение': 3, 'свадьба': 3, 'для': 2, 'коммерческий': 2, 'подержать': 2, 'сделка': 2, 'жилой': 2, 'заниматься': 2, 'дополнительный': 2, 'собственный': 1, 'семья': 1, 'профильный': 1, 'сдача': 1, 'со': 1, 'приобретение': 1, 'ремонт': 1, 'сыграть': 1, 'проведение': 1, '\n': 1})


### Категоризация данных

Выдлелил цели кредиты:  'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'операция': 4, 'свадьба': 3, 'строительство'

'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'свадьба': 3, 'ремонт':

Создание функции для категоризации

In [31]:
def purpose_category(purpose):
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'     
    return 'Прочее'        

In [32]:
data['purpose_cat'] = data['purpose'].apply(purpose_category, )

In [33]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,median_total_income,purpose_cat
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-1443.773383,177914.29425,недвижимость
1,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,-1443.773383,177914.29425,недвижимость
2,2,-4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,покупка недвижимости,-1443.773383,177914.29425,недвижимость
3,0,-529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,-1443.773383,177914.29425,недвижимость
4,1,-717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,строительство собственной недвижимости,-1443.773383,177914.29425,недвижимость


In [34]:
#Проверим категоризацию по столбцу цели. Получилось 4 категории
print(data['purpose_cat'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_cat, dtype: int64


In [35]:
print(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [36]:
# не может быть -1 ребенок в семье. Принимаем это за ошибку должен быть +1. 
data['children'] = data['children'].abs()

In [37]:
print(data['debt'].value_counts())

0    19713
1     1741
Name: debt, dtype: int64


## Шаг 3. Анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
report = data.groupby('children').agg({'debt': ['mean', 'count']})
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
children,,
20,10.53%,76
4,9.76%,41
2,9.45%,2052
1,9.17%,4855
3,8.18%,330
0,7.54%,14091
5,0.00%,9


In [39]:
# Из сформированной таблицы видно что с увеличением количество детей увеличиваеться процент просроченных платежей.
# Для налглядности сформируем таблицу сгруппированной по наличию детей и отсутсвию детей.

In [40]:
def child_group(value):
    if value == 0:
        return 0
    return 1

In [41]:
#report['child_group'] = report['children'].apply(child_group)

In [42]:
#print(type(report))

In [43]:
#report['child_group'] = report['children'].apply(child_group)

In [44]:
data['child_group'] = data['children'].apply(child_group)

In [45]:
report = data.groupby('child_group').agg({'debt': ['mean', 'count']})
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
child_group,,
1,9.21%,7363
0,7.54%,14091


In [46]:
# Здесь мы можем увидеть что наличие детей увеличивает процент просроченных платежей. 
# Разница между максимальным и минимальным % невозврата составляет на 1.67%

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
print(data['family_status_id'].value_counts())

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64


In [48]:
print(data['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [49]:
report = data.groupby('family_status').agg({'debt': ['mean', 'count']})
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
family_status,,
Не женат / не замужем,9.75%,2810
гражданский брак,9.35%,4151
женат / замужем,7.55%,12339
в разводе,7.11%,1195
вдовец / вдова,6.57%,959


In [50]:
# Интересные данные получились. Из приведенной таблицы видно что семейный статус влияет на возврат кредитов.
# Семья повышает ответсвенности у людей 
# статус не женат не замужем характеризует в большинстве случаем молодых людей и с возрастом ответсвенность увеличиваеться.
# Разница между максимальным и минимальным % невозврата составляет на 1.6%, 
# Проверим связь с возрастом

In [51]:
data['dob_years_group'] = pd.qcut(data['dob_years'],
                              q = [0, 0.25, 0.5, 0.75, 1], # процент разделение данных
                              labels = ['До 33', 'От 33 до 42', 'от 42 до 53', 'от 53'] #наименование 
                              )

In [52]:
data.groupby('dob_years_group')['dob_years'].describe()

,count,mean,std,min,25%,50%,75%,max
dob_years_group,,,,,,,,
До 33,5467.0,27.689592,5.085012,0.0,26.0,28.0,31.0,33.0
От 33 до 42,5284.0,38.016843,2.606487,34.0,36.0,38.0,40.0,42.0
от 42 до 53,5448.0,47.899780,3.151149,43.0,45.0,48.0,51.0,53.0
от 53,5255.0,59.966318,4.453051,54.0,56.0,59.0,63.0,75.0


In [53]:
report = data.groupby('dob_years_group').agg({'debt': ['mean', 'count']}) 
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
dob_years_group,,
До 33,10.88%,5467
От 33 до 42,8.57%,5284
от 42 до 53,7.32%,5448
от 53,5.59%,5255


In [54]:
#  Разница между максимальным и минимальным % невозврата 5,29%
#  Из этих данных мы видим что с возрастом доля просрочки уменьшаються.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
data['income_group'] = pd.qcut(data['total_income'],
                              q = [0, 0.25, 0.5, 0.75, 1], # процент разделение данных
                              labels = ['очень низкий', 'низкий', 'средний', 'высокий'] #наименование 
                              )

In [56]:
report = data.groupby('income_group').agg({'debt': ['mean', 'count']}) 
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
income_group,,
низкий,8.78%,5363
средний,8.50%,5363
очень низкий,8.00%,5364
высокий,7.18%,5364


In [57]:
# Разница между максимальным и минимальным % невозврата составляет на 3.18%, 
# все категории по кличеству заемщиков разделелились поровну.
# Влияние семейного статуса на просрочку незначительная. Относительно других категорий из представленого ДатаФрейма.

 Есть ли зависимость между целями и возвратом кредита в срок?

In [58]:
report = data.groupby('purpose_cat').agg({'debt': ['mean', 'count']}) 
report.columns = ['% невозврата', 'Кол-во заемщиков']
report = report.sort_values(by = '% невозврата', ascending = False)
report.style.format({'% невозврата':'{:.2%}'})

,% невозврата,Кол-во заемщиков
purpose_cat,,
автомобиль,9.36%,4306
образование,9.22%,4013
свадьба,8.00%,2324
недвижимость,7.23%,10811


In [59]:
# Были выявлены 4 цели кредита - автомобиль, образование, свадьба и недвижимость. 
# Разница между максимальным и минимальным значением просрочки составляет 2.13 %

Общая таблица данных по полученным данным в ходе анализа.

In [60]:
table = [
    ['children', '9,21%', '7,54%','1,67'],
    ['family_status', '9,75%', '6,57%', '3,18'],
    ['income', '8,78%', '7,18%', '1,6'],
    ['purpos', '9,36%', '7,23%', '2,13'],
    ['age', '10.88%', '5.59%', '5,29']
]
columns = ['показатель', 'max', 'min', 'разница']
result_table = pd.DataFrame(data = table, columns=columns)

In [61]:
result_table = result_table.sort_values('разница', ascending = False)

## Шаг 4. Общий вывод

После анализа предоставленных исходных данных. Можно сделать следующие выводы:
наибольшее влияние на платежеспособность заемщика оказывает симейный статус и возраст

  **Рабочие задачи**
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Общие результаты**
Больше всего на возврат в срок кредита оказывает влияние возраста. Чем старше тем меньше просрочек платежа. Следующая категория которая оказывает влияние "семейный статус", цели, дети и доход.  Семейный статус связан с возрастом. Среди молодых людей меньше людей связаны семьей. С возрастом сейменый статус соответсвенно меняеться. 
Цели кредита - меньше всего просрочек с целью "недвижимость".
Дети и доход. Чем больше у заемщика "свободных средств" (больше доход или меньше детей) тем меньше просрочек по кредиту.

**Рекомендации**
Внимательнее расматривать потенциального заемщика с детьми т.к. это повышает расходы; Молодежь до 30 лет возможно менее ответсвенны и нет стабильного дохода. 

In [62]:
result_table

,показатель,max,min,разница
4,age,10.88%,5.59%,"5,29"
1,family_status,"9,75%","6,57%","3,18"
3,purpos,"9,36%","7,23%","2,13"
0,children,"9,21%","7,54%","1,67"
2,income,"8,78%","7,18%","1,6"
